<a href="https://colab.research.google.com/github/jfink09/Financial-Modeling/blob/Screeners-and-Backtests/Stricter_Minervini_Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import tkinter as Tk
#from tkinter.filedialog import askopenfilenam

In [ ]:
# Install Yahoo Finance's library
!pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
# Initialize starting and ending timeframe
yf.pdr_override()
start = dt.datetime(2019,1,1)
now = dt.datetime.now()

In [ ]:
# Import Excel file containing every public company's ticker symbol
from google.colab import files
uploaded = files.upload()

Saving allstocks.xlsx to allstocks.xlsx


In [ ]:
# Read the Excel file
stocklist = pd.read_excel('allstocks.xlsx')

In [ ]:
Price = stocklist['Price']
RS_rating = stocklist['RS Rating']
Volume = stocklist['Vol. (1000s)']

In [ ]:
Price_over_15 = stocklist[Price > 15]
stocklist = Price_over_15

RS_rating_over_70 = stocklist[RS_rating > 70]
stocklist = RS_rating_over_70

Volume_over_400k = stocklist[Volume > 400]
stocklist = Volume_over_400k

Volume_over_400k

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


,Symbol,Price,Price $ Change,Price % Change,Comp. Rating,EPS Rating,RS Rating,Ind Grp RS,SMR Rating,Acc/Dis Rating,Spon Rating,Vol. % Change,Vol. (1000s)
2,AMBA,154.14,-4.51,-2.84,99.0,61.0,97.0,A+,C,A+,C,-22.0,519
3,AMD,105.06,-1.39,-1.31,99.0,99.0,80.0,A+,A,C+,B,-52.0,30623
5,ARES,77.73,0.50,0.65,99.0,78.0,93.0,A-,A,B+,B,-35.0,449
9,CCRN,22.83,0.38,1.69,99.0,64.0,98.0,A+,B,A-,C,112.0,673
10,CF,61.06,0.40,0.66,99.0,92.0,92.0,B+,C,A+,C,-22.0,2126
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9077,XLF,39.02,0.19,0.49,NaN,NaN,82.0,C+,NaN,D+,C,-21.0,38607
9087,XME,42.45,-0.40,-0.93,NaN,NaN,78.0,C+,NaN,B-,E,-13.0,3953
9094,XOP,105.37,2.58,2.51,NaN,NaN,94.0,C+,NaN,C+,E,4.0,8172
9101,XRT,90.84,-0.75,-0.82,NaN,NaN,78.0,C+,NaN,D-,C,-43.0,1575


In [ ]:
# Create a list to add specific data
technical_list = pd.DataFrame(columns=['Stock', "RS_Rating", "50 Day MA", "150 Day Ma", "200 Day MA", "52 Week Low", "52 week High", "21 Day EMA"])

In [ ]:
for i in stocklist.index:

  stock = str(stocklist['Symbol'][i])
  RS_rating = stocklist['RS Rating'][i]

  try:

    df = pdr.get_data_yahoo(stock, start, now)

    #print("Checking "+stock+"...")

  except Exception:

      print("No data on "+stock)

In [ ]:
# Array of each SMA used 

sma = [50, 150, 200]

for x in sma:

  sma = x

  # Calculate the three SMAs
  df["SMA_"+str(sma)] = round(df.iloc[:,4].rolling(window=sma).mean(),2)


# Get most recent data from Yahoo Finance

currentClose = df['Adj Close'][-1]
moving_average_50 = df['SMA_50'][-1]
moving_average_150 = df['SMA_150'][-1]
moving_average_200 = df['SMA_200'][-1]
week_low_52 = min(df['Adj Close'][-260:])
week_high_52 = max(df['Adj Close'][-260:])

try:

  # Get 200SMA moving in an uptrend for a month
  moving_average_200_20past = df['SMA_200'][-20]

except Exception:

  moving_average_200_20past = 0

In [ ]:
# Current price over 150sma over 200sma

if(currentClose > moving_average_150 and currentClose > moving_average_200):
  condition_1 = True

else:
  condition_1 = False


# 150sma over 200sma

if(moving_average_150 > moving_average_200):
  condition_2 = True

else:
  condition_2 = False


# 200sma in uptrend for more than 1 month

if(moving_average_200 > moving_average_200_20past):
  condition_3 = True

else:
  condition_3 = False


# 50sma over 150sma and 50sma over 200sma

if(moving_average_50 > moving_average_150 and moving_average_50 > moving_average_200):
  condition_4 = True

else:
  condition_4 = False


# Current price over 50sma

if(currentClose > moving_average_50):
  condition_5 = True

else: 
  condition_5 = False


# Current price is 30% higher than the 52-week low

if(currentClose > (1.3*week_low_52)):
  condition_6 = True

else: 
  condition_6 = False


# Current price is within 25% from its 52-week high

if(currentClose >= (0.75*week_high_52)):
  condition_7 = True

else: 
  condition_7 = False


# IBD RS rating over 70

if(RS_rating > 70):
  condition_8 = True

else: 
  condition_8 = False

if(currentClose > moving_average_21):
  condition_9 = True

else:
  condition_9 = False


# If each condition is met, create a list of the stocks that meet all criteria

if(condition_1 and condition_2 and condition_3 and condition_4 and condition_5 and condition_6 and condition_7 and condition_8):
  technical_list = technical_list.append({'Stock': stock, "RS_Rating": RS_rating, "50 Day MA": moving_average_50, "150 Day Ma": moving_average_150, 
                                  "200 Day MA": moving_average_200, "52 Week Low": week_low_52, "52 week High": week_high_52}, ignore_index=True)


print(technical_list)

Empty DataFrame
Columns: [Stock, RS_Rating, 50 Day MA, 150 Day Ma, 200 Day MA, 52 Week Low, 52 week High, 21 Day EMA]
Index: []
